In [73]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
print(np.__version__)

2.11.0
1.20.3


In [74]:
from PIL import Image
import glob
import os

print("PIL 라이브러리 import 완료!")

PIL 라이브러리 import 완료!


## 데이터 준비하기

### 데이터 불러오기 + Resize 하기

In [75]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = ('/Users/user/aiffel/project/rock_scissor_paper/scissor')
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

1115  images to be resized.
1115  images resized.
가위 이미지 resize 완료!


In [76]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = ('/Users/user/aiffel/project/rock_scissor_paper/rock')
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")

1293  images to be resized.
1293  images resized.
바위 이미지 resize 완료!


In [77]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = ('/Users/user/aiffel/project/rock_scissor_paper/paper')
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

980  images to be resized.
980  images resized.
보 이미지 resize 완료!


* 가위, 바위, 보 데이터를 읽을 수 있는 load_dat()함수 만들기

In [78]:
import numpy as np

def load_data(img_path, number_of_data=3388):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = ('/Users/user/aiffel/project/rock_scissor_paper')
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 3388 입니다.
x_train shape: (3388, 28, 28, 3)
y_train shape: (3388,)


## 딥러닝 네트워크 설계하기

In [79]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))



model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 26, 26, 64)        1792      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 13, 13, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 11, 11, 128)       73856     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 5, 5, 128)        0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 3200)              0         
                                                                 
 dense_12 (Dense)            (None, 64)               

In [80]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 모델 훈련
model.fit(x_train_norm, y_train , epochs=15)

Epoch 1/15
106/106 [==============================] - 2s 13ms/step - loss: 0.9167 - accuracy: 0.5549
Epoch 2/15
106/106 [==============================] - 1s 13ms/step - loss: 0.4651 - accuracy: 0.8247
Epoch 3/15
106/106 [==============================] - 1s 13ms/step - loss: 0.2060 - accuracy: 0.9324
Epoch 4/15
106/106 [==============================] - 1s 13ms/step - loss: 0.1053 - accuracy: 0.9681
Epoch 5/15
106/106 [==============================] - 1s 13ms/step - loss: 0.0572 - accuracy: 0.9861
Epoch 6/15
106/106 [==============================] - 1s 13ms/step - loss: 0.0401 - accuracy: 0.9891
Epoch 7/15
106/106 [==============================] - 1s 13ms/step - loss: 0.0250 - accuracy: 0.9938
Epoch 8/15
106/106 [==============================] - 1s 13ms/step - loss: 0.0149 - accuracy: 0.9973
Epoch 9/15
106/106 [==============================] - 1s 13ms/step - loss: 0.0110 - accuracy: 0.9982
Epoch 10/15
106/106 [==============================] - 1s 13ms/step - loss: 0.0075 - accura

### Test data 불러오기 

In [81]:
def load_test(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=600   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color, dtype=np.int32).reshape(number_of_data, img_size, img_size, color)
    labels=np.zeros(number_of_data, dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    return imgs, labels

In [82]:
test_image_dir_path = ('/Users/user/aiffel/project/rock_scissor_paper/test')
print(test_image_dir_path)

# scissor
t_s_images=glob.glob(test_image_dir_path + "/scissor/*.jpg")  
target_size=(28,28)
for img in t_s_images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("가위 이미지 resize 완료!")

# rock
t_r_images=glob.glob(test_image_dir_path + "/rock/*.jpg")  
target_size=(28,28)
for img in t_r_images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img, "JPEG")
 
print("바위 이미지 resize 완료!")

# paper
t_p_images=glob.glob(test_image_dir_path + "/paper/*.jpg")  
target_size=(28,28)
for img in t_p_images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img, "JPEG")
 
print("보 이미지 resize 완료!")


(x_test, y_test)=load_test(test_image_dir_path)
x_test_norm = x_test/255.0

print(x_test.shape)

/Users/user/aiffel/project/rock_scissor_paper/test
가위 이미지 resize 완료!
바위 이미지 resize 완료!
보 이미지 resize 완료!
(600, 28, 28, 3)


###  모델 테스트 하기

In [83]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

19/19 - 0s - loss: 6.7558 - accuracy: 0.4917 - 150ms/epoch - 8ms/step
test_loss: 6.755769729614258 
test_accuracy: 0.49166667461395264


### 느낀점...

데이터를 점점 늘리면 늘릴수록 점수가 낮아졌다...
느낀점은 가위바위보 이미지의 방향이나 컴퓨터가 인식하려고 할때 방해 요소가 있으면 그 데이터의 양이 어마어마하게 방대하지 않은 한 정확하게 인식하지 못한다는 느낌이다. 
중간에 데이터의 수를 그저 많이 늘려보다가 점수가 너무 나오지 않아 너무 다르게 생긴 데이터들을 조금 삭제해 보았다. 이렇게 했을 때, 점수가 높아진다면 흠....그래도 정상적으로 작동을 하는구나라는 생각에 그렇게 했는데, 점수가 조금은 높게 나왔다. 딥러닝은 너무 어렵다. 
